# Management and Analysis of Physics Dataset - mod.B

## Final project: Streaming processing of cosmic rays using Drift Tubes detectors

The goal of this project is to reproduce a real-time processing of real data collected in a particle physics detector and publish the results in a dashboard for live monitoring.

### Students:
* Conforto Filippo (2021856)
* Domenichetti Lorenzo (missing)
* Faorlin Tommaso (2021857)

## Producer notebook

In [2]:
import os
import pandas as pd
import ssl
import json

from kafka import KafkaProducer
import time

ssl._create_default_https_context = ssl._create_unverified_context

New topic

In [3]:
from kafka.admin import KafkaAdminClient, NewTopic
KAFKA_BOOTSTRAP_SERVERS='10.67.22.100:9092'
kafka_admin = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    )

In [4]:
kafka_admin.list_topics()

['topic_stream',
 '__consumer_offsets',
 'topic_test',
 'topic_result',
 'topic_results']

In [5]:
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [6]:
i=19
url = f"https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/MAPD_miniDT_stream/data_0000{i}.txt"
df = pd.read_csv(url)
rate = 1000

In [36]:
for i in range(0,81):
    i = str(i).zfill(2)
    url = f"https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/MAPD_miniDT_stream/data_0000{i}.txt"
    df = pd.read_csv(url)
    for j in range(0,df.shape[0]):
        jj = df.iloc[j].to_dict()
        for key in ['HEAD', 'FPGA', "TDC_CHANNEL", "ORBIT_CNT", "BX_COUNTER"]:
            jj[key] = int(jj[key])
        producer.send('topic_stream', json.dumps(jj).encode('utf-8'))
        time.sleep(0.0004)
    producer.flush()

KeyboardInterrupt: 